In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Fashion Boutique dataset/fashion_boutique_dataset.csv")
df.head(
)

,product_id,category,brand,season,size,color,original_price,markdown_percentage,current_price,purchase_date,stock_quantity,customer_rating,is_returned,return_reason
0,FB000001,Outerwear,Zara,Spring,XL,Red,196.01,0.0,196.01,2025-07-05,37,3.0,False,NaN
1,FB000002,Tops,Uniqlo,Winter,L,Pink,119.64,0.0,119.64,2025-08-06,2,2.5,False,NaN
2,FB000003,Accessories,Uniqlo,Winter,NaN,Black,33.80,0.0,33.80,2025-08-06,22,4.3,False,NaN
3,FB000004,Shoes,Uniqlo,Spring,XL,Black,75.36,0.0,75.36,2025-07-07,48,2.6,False,NaN
4,FB000005,Tops,Banana Republic,Winter,XL,Black,105.02,0.0,105.02,2025-08-06,10,NaN,False,NaN


In [7]:
import sqlite3
import pandas as pd

# Load CSV
df = pd.read_csv("/content/drive/MyDrive/Fashion Boutique dataset/fashion_boutique_dataset.csv")

# Create SQLite DB
conn = sqlite3.connect("mydata.db")
df.to_sql("mytable", conn, if_exists="replace", index=False)
conn.commit()

In [5]:
!pip install langgraph
!pip install langchain_community
!pip install genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requ

In [6]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
import os
from langchain_core.runnables import RunnableLambda
import pandas as pd
import sqlite3
from google import genai

In [12]:
# Setup
# ======================
with open("/content/drive/MyDrive/Colab Notebooks/API_KEY.txt", "r") as f:
    API_KEY = f.read()
client = genai.Client(api_key=API_KEY)

# Connect to SQLite
conn = sqlite3.connect("mydata.db")

# ======================
# Define Shared State
# ======================
class AgentState(TypedDict, total=False):
    user_input: str
    intent: str
    sql_query: Optional[str]
    result: Optional[object]
    error: Optional[str]
    attempts: int
    max_attempts: int
    final_answer: str


class AgentwithLangGraph:
    def __init__(self, max_attempts=3):
        self.max_attempts = max_attempts
        self.graph = self._build_workflow()

    # ======================
    # Define Agent Functions
    # ======================
    def flow_manager(self, state: AgentState):
        state["intent"] = "query"
        state["attempts"] = 0
        state.setdefault("max_attempts", 3)
        return state

    def nl_to_sql(self, state: AgentState):
        schema = """
        Table: mytable
        Columns:
        - product_id (INTEGER) : Unique product identifier
        - category (TEXT) : Product category
        - brand (TEXT) : Brand of product
        - season (TEXT) : Season (e.g., Summer, Winter)
        - size (TEXT) : Size of product
        - color (TEXT) : Product color
        - original_price (REAL) : Original product price
        - markdown_percentage (REAL) : Discount percentage
        - current_price (REAL) : Final price after discount
        - purchase_date (DATE) : Date of purchase
        - stock_quantity (INTEGER) : Number of items in stock
        - customer_rating (REAL) : Customer rating (1–5)
        - is_returned (BOOLEAN) : Whether the item was returned
        - return_reason (TEXT) : Reason for return
        """
        prompt = f"""
        You are a **SQL Converter Agent**.
        Your job is to convert natural language questions into valid **SQLite SQL queries**.

        - Only use the table: mytable
        - Use this schema:
        {schema}

        Rules:
        - Output ONLY the SQL query (no explanations, no markdown formatting).
        - Use correct SQLite syntax.
        - Always SELECT only the needed columns.
        - If aggregation is asked (average, sum, count, etc.), use the appropriate SQL functions.
        - If ordering is asked (top, highest, lowest, etc.), use ORDER BY and LIMIT.
        - If filtering is asked (specific category, brand, season, etc.), use WHERE clause.
        - Use column names exactly as provided.

        Question: "{state['user_input']}"

        SQL Query:
        """
        response = client.models.generate_content(
            model="gemini-2.5-flash", contents=prompt
        )
        state["sql_query"] = response.text.strip()
        return state

    def sql_executor(self, state: AgentState):
        try:
            result = pd.read_sql_query(state["sql_query"], conn)
            state["result"] = result
            state["error"] = None
        except Exception as e:
            state["error"] = str(e)
        return state

    def query_repair(self, state: AgentState):
        state["attempts"] += 1
        if state["attempts"] >= state["max_attempts"]:
            state["error"] = f"Max attempts reached ({state['max_attempts']}). Last error: {state['error']}"
            return state

        prompt = f"""
        You are a **SQL Repair Agent**.
        The following query failed. Fix it for SQLite.

        User Question:
        {state['user_input']}

        Failed SQL Query:
        {state['sql_query']}

        Error Message:
        {state['error']}

        Table Schema:
        - product_id, category, brand, season, size, color, original_price,
          markdown_percentage, current_price, purchase_date, stock_quantity,
          customer_rating, is_returned, return_reason

        Output only the corrected SQL query.
        """
        response = client.models.generate_content(
            model="gemini-2.5-flash", contents=prompt
        )
        state["sql_query"] = response.text.strip()
        return state

    def answer_formatter(self, state: AgentState):
        if state.get("error"):
            state["final_answer"] = f" Query failed: {state['error']}"
            return state

        result = state.get("result")
        user_input = state.get("user_input")

        if result is None or result.empty:
            state["final_answer"] = f"Sorry, I couldn’t find any data for your query: '{user_input}'"
            return state

        table_text = result.to_string(index=False)

        prompt = f"""
        You are an **Answer Formatter Agent**.
        Your job is to take a raw SQL result table and explain it in clear, natural language.

        Rules:
        - Do NOT output the raw table unless absolutely necessary.
        - Summarize the key findings in plain English.
        - Keep the explanation short, precise, and conversational.
        - If numbers are present, mention them naturally.
        - Stay faithful to the data (no hallucinations).

        User Question:
        {user_input}

        SQL Result Table:
        {table_text}

        Final Answer:
        """
        response = client.models.generate_content(
            model="gemini-2.5-flash", contents=prompt
        )
        state["final_answer"] = response.text.strip()
        return state

    def should_retry(self, state: AgentState):
        if state.get("attempts", 0) >= state.get("max_attempts", 3):
            return "end"
        if state.get("error") is None:
            return "generate"
        return "retry"

    def format_result(self, state: AgentState):
        state["final_answer"] = (
            f"Failed after {state['attempts']} attempts. Last error: {state['error']}"
        )
        return state

    # ======================
    # Build LangGraph
    # ======================
    def _build_workflow(self):
        graph = StateGraph(AgentState)

        graph.add_node("flow_manager", RunnableLambda(self.flow_manager))
        graph.add_node("nl_to_sql", RunnableLambda(self.nl_to_sql))
        graph.add_node("sql_executor", RunnableLambda(self.sql_executor))
        graph.add_node("query_repair", RunnableLambda(self.query_repair))
        graph.add_node("format_result", RunnableLambda(self.format_result))
        graph.add_node("answer_formatter", RunnableLambda(self.answer_formatter))

        graph.set_entry_point("flow_manager")

        graph.add_edge("flow_manager", "nl_to_sql")
        graph.add_edge("nl_to_sql", "sql_executor")
        graph.add_edge("query_repair", "sql_executor")
        graph.add_edge("answer_formatter", END)
        graph.add_edge("format_result", END)

        graph.add_conditional_edges(
            "sql_executor",
            self.should_retry,
            {
                "end": "format_result",
                "retry": "query_repair",
                "generate": "answer_formatter",
            },
        )
        return graph.compile()

    def run(self, question: str) -> str:
        initial_state = {"user_input": question, "max_attempts": self.max_attempts}
        final_state = self.graph.invoke(initial_state)
        return final_state.get("final_answer", "No answer found")


# ======================
# Example Run
# ======================
app = AgentwithLangGraph(max_attempts=2)
user_question = input("Enter your question: ")
final_answer = app.run(user_question)
print("Final Answer:", final_answer)

Enter your question: which brand is returned least number of times
Final Answer: Based on our data, the brand "Mango" was returned the least number of times.
